<a href="https://colab.research.google.com/github/frontenddeveloper4web/Chatbot-with-OpenAI-/blob/main/Text_Emotions_Classification_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [5]:
texts  = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [6]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [7]:
sequences = tokenizer.texts_to_sequences(texts)

In [8]:
padded_sequences

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]], dtype=int32)

In [9]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [10]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [11]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

In [12]:
one_hot_labels[0]

array([0., 0., 0., 0., 1., 0.], dtype=float32)

In [14]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 17s 41ms/step - loss: 1.3350 - accuracy: 0.4892 - val_loss: 0.8588 - val_accuracy: 0.7063
Epoch 2/10
400/400 [==============================] - 17s 42ms/step - loss: 0.3705 - accuracy: 0.8823 - val_loss: 0.5450 - val_accuracy: 0.8172
Epoch 3/10
400/400 [==============================] - 15s 39ms/step - loss: 0.0668 - accuracy: 0.9829 - val_loss: 0.5729 - val_accuracy: 0.8300
Epoch 4/10
400/400 [==============================] - 15s 39ms/step - loss: 0.0274 - accuracy: 0.9937 - val_loss: 0.6133 - val_accuracy: 0.8238
Epoch 5/10
400/400 [==============================] - 16s 40ms/step - loss: 0.0186 - accuracy: 0.9963 - val_loss: 0.6581 - val_accuracy: 0.8272
Epoch 6/10
400/400 [==============================] - 17s 41ms/step - loss: 0.0160 - accuracy: 0.9965 - val_loss: 0.6640 - val_accuracy: 0.8216
Epoch 7/10
400/400 [==============================] - 16s 40ms/step - loss: 0.0137 - accuracy: 0.9973 - val_loss: 0.7318 - val_accuracy:

In [16]:
input_text = "She didn't come today because she lost her dog yesterday!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 [==============================] - 0s 20ms/step
sadness


In [17]:
input_text = "i am ever feeling nostalgic about the fireplace i will know that it is still on the property"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 [==============================] - 0s 28ms/step
love
